# Introduction

This notebook covers the **first** step of SQLAlchemy.

It covers topics in this order:

1. logging (because of a quick in the system).
2. Engine (Dialect and Driver)

# Logging
SQLAlchemy is pretty good with *what* it logs, but it has a quirk if you start logging after creating an engine.

Logging is important because some queries will cause multiple executions (mainly in ORM queries).<br>
These queries aren't shown when using `str(query)`.<br>
This usually happens in a table join statement.

The text `[generated in X.Ys]` refers to the time it took to generate a valid SQL string from code.

The text `[cached since XXs ago]` comes up in logging a lot.
This refers to the cache for the QueryBuilder, in which the query is serialized to a valid SQL string.
It does **NOT** refer to cached database results.

*See also: [SQL Compilation Caching at docs.sqlalchemy.org](https://docs.sqlalchemy.org/en/20/core/connections.html#sql-compilation-caching)*

The logger here is outputting to 'Standard Output'.
Jupyter Notebooks don't relay messages in perfect real-time. That means that when a message is sent to `stdout` and `stderr` at nearly the same time, they might arrive out of order.

Keeping both on `stdout` keeps clarity about what happens first.


In [ ]:
import logging
import sys

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.WARN)

logger = logging.getLogger('sqlalchemy.engine')
logger.setLevel(logging.DEBUG)
logger.addHandler(handler)

The code below adds a basic context decorator to more easily turn logging on/off when it's actually interesting.
```python
with logs():
    ...  # SQLAlchemy code goes here
```

| Level | Data |
|-------|-------|
| logging.INFO | SQL Query |
| logging.DEBUG | Returned rows |

In [ ]:
from contextlib import contextmanager

@contextmanager
def logs(level=logging.INFO):
    state = handler.level
    handler.setLevel(level)
    try:
        yield
    finally:
        handler.setLevel(state)

# Engine 

SQLAlchemy differentiates between Language(dialect) and Technical Implementation(driver).

Putting those two together creates the Engine, a configuration block (or facade) to abstract connection setup.
The engine also allows enabling of certain features in a unified way.


## Dialect
The Dialect is flavor of SQL being used, the specific syntax.

SQLAlchemy supports a lot of dialects right out of the box, ranging from SQLite to PostgreSQL and (MS)SQL.
Companies might implement their own dialect (probably via a Python package) to improve support.

For example: Some flavors use `OFFSET, LIMIT` to do pagination, while others use `FETCH NEXT` or `TOP`.

In most cases, SQLAlchemy will have you covered by default.

## Driver
The driver takes care of the provider specific technical details.

If you're using an SQL server, it takes care of the TCP network details. For SQLite, it will take care of file access.

The driver usually has to be installed, but SQLite is supported out of the box.

## URL
The concepts of Dialect and Driver are neat, but how are they applied? SQLAlchemy uses a URL-like structure for this.

For those familiar with HTTP Basic Auth, it's roughly that. For those unfamilair with it, it is best demonstrated by creating that string programmatically.

The only thing that needs explanation is how a to specify dialect and driver.
This is usually formatted as `<dialect>+<driver>`.
The below demonstrates this with the PostgreSQL dialect, using the `pg8000` driver.

In [ ]:
import sqlalchemy as sa

print('In-memory (SQLite):', sa.URL.create('sqlite'))
print('Simple (MySQL):', sa.URL.create('mysql', host='127.0.0.1'))
# Note: the password will be automatically censored since SQLAlchemy 2.0
print('Complex (Postgres w/ pg8000 driver):', sa.URL.create('postgresql+pg8000', username='root', password='alchemy', 
                                                            host='127.0.0.1', database='mytest'))

----------
Although a basic string can be used, ``URL.create`` can prevent a lot of issues.

Some DBMS might require more complex connection strings.<br>
For those, look at the notebooks containing those names.

## Code

In [ ]:
import sqlalchemy as sa

# First 'connect' to a database.
# The engine takes care of the Dialect (language) and Driver (communication protocol).
# This line creates an in-memory database (using SQLite)
# This is nice, because it means the notebooks won't cross-contaminate.
engine = sa.create_engine('sqlite://')

# Alternatively:
# engine = sa.create_engine(sa.URL('sqlite'))

con = engine.connect()

In [ ]:
# Example query to text the connection, and logging.
with logs():
    query = sa.text('SELECT "John" as FirstName, "Doe" as LastName;')
    for row in con.execute(query):
        print(*row)